# Installs and Imports

## Installs

In [2]:
%pip install neo4j

     ---------------------------------------- 90.8/90.8 KB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 503.5/503.5 KB 7.8 MB/s eta 0:00:00
Using legacy 'setup.py install' for neo4j, since package 'wheel' is not installed.
  Running setup.py install for neo4j: started
  Running setup.py install for neo4j: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\nelso\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [ ]:
%pip install -i https://test.pypi.org/simple/ mdb-tools

## Imports

In [5]:
from neo4j import GraphDatabase, basic_auth
import mdb_tools.mdb_tools as mdb

# Database connection

URL, username, and password for database connection

In [6]:
# MDB sandbox
url = "bolt://localhost:7687" #"<URL for database>"
user = "neo4j" #"<Username for database>"
password = "noble-use-dairy" #"<Password for database>"
driver = GraphDatabase.driver(url, auth=(user, password))

In [3]:
dir(mdb)

['GraphDatabase',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'basic_auth',
 'check_term_exists',
 'create_concept',
 'create_object_relationship',
 'create_predicate',
 'create_represents_relationship',
 'create_represents_relationship_from_nanoid',
 'create_subject_relationship',
 'create_term',
 'create_term_from_nanoid',
 'detach_delete_concept',
 'detach_delete_predicate',
 'generate',
 'generate_unique_nanoid',
 'get_concept',
 'get_terms',
 'link_concepts_to_predicate',
 'link_two_terms',
 'make_nano',
 'merge_two_concepts']

# Linking Terms


### Check existence of Term

In [ ]:
# generate test term
test_term = {
    "value": "Cancer",
    "origin_name": "GDC"
}

In [ ]:
# should return "Term with that value found in DB"
with driver.session() as session:
  term_exists = session.read_transaction(mdb.check_term_exists, test_term)
  if term_exists:
    print("Term with that value found in DB")
  else:
    print("Term with that value not found in DB")
driver.close()

### Find existing Concept from Term

In [ ]:
# generate test term
test_term = {
    "value": "Lung",
    "origin_name": "NCIt"
}

In [ ]:
# should return "4jtzA3"
with driver.session() as session:
    test_term_concepts = session.read_transaction(mdb.get_concept, test_term)
    print(test_term_concepts)
driver.close()

### Create new Term

In [ ]:
# generate test term
test_term = {
    "value": "Lung",
    "origin_name": "NDC"
}

In [ ]:
# should add term with value "Lung" and origin_name "NDC" and check it exists afterwards
with driver.session() as session:
    session.write_transaction(mdb.create_term, test_term)
    
    term_exists = session.read_transaction(mdb.check_term_exists, test_term)
    if term_exists:
        print("Term with that value found in DB")
    else:
        print("Term with that value not found in DB")
driver.close()

### Generate unique nanoid

In [81]:
with driver.session() as session:
  session.read_transaction(mdb.generate_unique_nanoid)
driver.close()

hjpBQV


### Create Concept

In [ ]:
# generate test Concept
with driver.session() as session:
    test_nano = session.read_transaction(mdb.generate_unique_nanoid)
driver.close()

test_concept = {
    "nanoid": test_nano
}

In [ ]:
with driver.session() as session:
    session.write_transaction(mdb.create_concept, test_concept)
driver.close()

Created new Concept with nanoid: E1PqVF


### Link Term and Concept

In [44]:
# test term and concept created above
print(test_term)
print(test_concept)

{'value': 'Lung', 'origin_name': 'NDC'}
{'nanoid': 'E1PqVF'}


In [ ]:
with driver.session() as session:
    session.write_transaction(mdb.create_represents_relationship, test_term, test_concept)
driver.close()

Term with value: Lung and origin: NDC now represents Concept with nanoid: E1PqVF


## Link Two Terms

In [18]:
# both terms exist & connected via concept
mdb.link_two_terms("Epithelioma, benign", "Epithelial tumor, benign")

Epithelioma, benign and Epithelial tumor, benign are already connected via Concept H6X2Ph


In [ ]:
# both terms exist but neither have concept representing them
test_term_1 = {
    "value": "Cancer",
    "origin_name": "BentoTailorX"
}

test_term_2 = {
    "value": "Cancer",
    "origin_name": "GDC"
}

mdb.link_two_terms(test_term_1, test_term_2)

Created new Concept with nanoid: aE7NQM
Term with value: Cancer and origin: BentoTailorX now represents Concept with nanoid: aE7NQM
Term with value: Cancer and origin: GDC now represents Concept with nanoid: aE7NQM


In [19]:
# both terms exist & NOT connected via concept
mdb.link_two_terms("Spindle cell carcinoma, NOS", "Sarcomatoid Carcinoma")

Created new Concept with nanoid: Zith4U
Spindle cell carcinoma, NOS Term now represents Zith4U Concept
Sarcomatoid Carcinoma Term now represents Zith4U Concept


In [20]:
# one term exists & already has concept
mdb.link_two_terms("Minimally Invasive Lung Adenocarcinoma", "Alveolar adenocarcinoma")

Created new Term with value: Alveolar adenocarcinoma
Alveolar adenocarcinoma Term now represents p2IbFv Concept


In [21]:
# one term exists & doesn't have concept yet
mdb.link_two_terms("Undifferentiated Carcinoma", "Carcinoma, anaplastic, NOS")

Created new Concept with nanoid: dBSPWS
Created new Term with value: Carcinoma, anaplastic, NOS
Undifferentiated Carcinoma Term now represents dBSPWS Concept
Carcinoma, anaplastic, NOS Term now represents dBSPWS Concept


In [22]:
# neither term exists
mdb.link_two_terms("Epithelioma, malignant", "Carcinoma")

Created new Term with value: Epithelioma, malignant
Created new Term with value: Carcinoma
Created new Concept with nanoid: 4CneRp
Epithelioma, malignant Term now represents 4CneRp Concept
Carcinoma Term now represents 4CneRp Concept


# Linking Concepts


When two existing Concept nodes are deemed synonymous, there are two primary ways to approach linking them together. The first way to link the synonymous Concepts would be via a Predicate node with the an "exactMatch" handle. This method maintains the exisiting Concept & Term structure while adding to it, allowing queries already in use to continue to work. 

The second way is simply merging the two so they are represented by the same Concept node. With this approach, the Terms linked to each Concept would then be linked to the new merged Concept instead. They could be merged under one of the exisiting Concepts or a new Concept could be created and the old two removed. This method would invalidate existing queries using relevant Concepts & Terms.

### Create Predicate

In [ ]:
with driver.session() as session:
    test_nano = session.read_transaction(mdb.generate_unique_nanoid)
driver.close()

test_predicate = {
    "handle": "exactMatch",
    "nanoid": test_nano
}

In [ ]:
with driver.session() as session:
    session.write_transaction(mdb.create_predicate, test_predicate)
driver.close()

Created new Predicate with handle: exactMatch and nanoid: 1N5WiS


### Link two Concepts to a Predicate

General pattern: (c1:concept)<-[:has_subject]-(p:predicate {handle:“exactMatch”})-[:has_object]->(c2:concept)

In [ ]:
# these Concepts both represent Terms with value: 'Lung' in the MDB.
test_concept_1 = {"nanoid": "E1PqVF"}
test_concept_2 = {"nanoid": "UWs6Di"}

mdb.link_concepts_to_predicate(test_concept_1, test_concept_2)

Created new Predicate with handle: exactMatch and nanoid: Bpp7ab
Created has_subject relationship between source Predicate with handle: exactMatch and nanoid: Bpp7ab and destination Concept with nanoid: E1PqVF
Created has_object relationship between source Predicate with handle: exactMatch and nanoid: Bpp7ab and destination Concept with nanoid: UWs6Di


### Merge two Concepts

In [ ]:
# see Terms attached to test Concept
test_concept = {"nanoid": "qkqPeC"}

with driver.session() as session:
    terms = session.read_transaction(mdb.get_terms, test_concept)
    print(terms)
driver.close()

In [81]:
# delete test Concept
with driver.session() as session:
    session.write_transaction(mdb.detach_delete_concept, test_concept)
driver.close()

In [87]:
# generate new Concepts
with driver.session() as session:
    test_nano_1 = session.read_transaction(mdb.generate_unique_nanoid)
    test_nano_2 = session.read_transaction(mdb.generate_unique_nanoid)
    test_concept_1 = {"nanoid": test_nano_1}
    test_concept_2 = {"nanoid": test_nano_2}
    session.write_transaction(mdb.create_concept, test_concept_1)
    session.write_transaction(mdb.create_concept, test_concept_2)
driver.close()

Created new Concept with nanoid: qkqPeC
Created new Concept with nanoid: nDHYF0


In [89]:
# link new Concepts to Terms above (2 Terms to 1st Concept, 1 Term to 2nd Concept)
with driver.session() as session:
    test_term_1 = {"value": "Nelson", "origin_name": "NDC"}
    test_term_2 = {"value": "Nelsinghouse", "origin_name": "NDC"}
    test_term_3 = {"value": "Nelly", "origin_name": "NDC"}

    session.write_transaction(mdb.create_represents_relationship, 
                            test_term_1, test_concept_1)
    session.write_transaction(mdb.create_represents_relationship, 
                            test_term_2, test_concept_1)
    session.write_transaction(mdb.create_represents_relationship, 
                            test_term_3, test_concept_2)
driver.close()

Term with value: Nelson and origin: NDC now represents Concept with nanoid: qkqPeC
Term with value: Nelsinghouse and origin: NDC now represents Concept with nanoid: qkqPeC
Term with value: Nelly and origin: NDC now represents Concept with nanoid: nDHYF0


In [90]:
# merge Concepts into one
test_concept_1 = {"nanoid": "qkqPeC"}
test_concept_2 = {"nanoid": "nDHYF0"}

mdb.merge_two_concepts(test_concept_1, test_concept_2)

Term with value: Nelly and origin: NDC now represents Concept with nanoid: qkqPeC
